In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

agent = create_agent(
    model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite"),
    tools=tools,
    system_prompt=prompt
)

In [6]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='eb3da245-9ea8-4dbf-b104-19571c77aa76'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_web', 'arguments': '{"query": "langchain-mcp-adapters library"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ca373-6932-7902-ba7e-eb0577605770-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters library'}, 'id': '8d910557-8b3d-44d6-b427-b6be9ae876e7', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 194, 'output_tokens': 22, 'total_tokens': 216, 'input_token_details': {'cache_read': 0}}),
              ToolMessage(content=[{'type': 'text', 'text': '{\n  "query": "langchain-mcp-adapters library",\n  "follow_up_questions": null,\n  "answer": null,\n  "images":

## Online MCP

In [8]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [9]:
agent = create_agent(
    model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite"),
    tools=tools,
)

In [11]:
question = HumanMessage(content="What time is it in IST?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it in IST?', additional_kwargs={}, response_metadata={}, id='989345b7-2549-49a9-8ba8-dddbf2c6caa0'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"timezone": "Asia/Kolkata"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ca379-7850-7c43-abf3-fa802502f0f6-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Kolkata'}, 'id': '0f28ed0e-0668-41bc-bb43-32bb77aa128c', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 252, 'output_tokens': 21, 'total_tokens': 273, 'input_token_details': {'cache_read': 0}}),
              ToolMessage(content=[{'type': 'text', 'text': '{\n  "timezone": "Asia/Kolkata",\n  "datetime": "2026-02-28T14:29:35+05:30",\n  "day_of_week": "Saturday",\n  "is_dst": false\n}', 'id': 'lc_0134f448-611a